In [1]:
import pandas as pd

UCR data is submitted voluntarily by law enforcement agencies and may vary in completeness across jurisdictions and years. This variation can result in gaps in monthly reporting, making it necessary to apply weighting or imputation procedures to ensure consistent year-over-year comparisons, especially when aggregating data to the county or state level.

UCRP Data Unit of Observation
- 2016 County
- 2020 Agency
(will follow 2016 methodology on 2020 to get 2020 county level estimates)

https://www.icpsr.umich.edu/web/ICPSR/studies/38791 (link for 2020 data too large for github)



## **Uniform Crime Reporting Program Data: County-Level Detailed Offense Data 2016**
Within each county, data from agencies reporting 3 to 11 months of information were weighted to yield 12-month equivalents. Data for agencies reporting less than three months of data were replaced with data estimated by rates calculated from agencies reporting 12 months of data located in the agency's geographic stratum within its state. Secondly, a new Coverage Indicator was created to provide users with a diagnostic measure of aggregated data quality in a particular county. Data from agencies reporting only statewide figures were allocated to the counties in the state in proportion to each county's share of the state population.
However, due to the structure of the data received from the Federal Bureau of Investigation (FBI), estimations could not be produced for agencies reporting zero months in the crimes reported files (Parts 4 and 8).

Note:
* UCR county-level files are not official FBI UCR releases and are being provided for research purposes only
* Limited crimes reported data were available for Mississippi and South Dakota.
* Coverage Indicator is calculated as follows:
CI_x = 100 * ( 1 - SUM_i { [ORIPOP_i/COUNTYPOP] * [ (12 - MONTHSREPORTED_i)/12 ] } )
where CI = Coverage Indicator
x = county
i = ORI within county


In [49]:
columns = {'FIPS_ST': 'uint8', # FIPS State code
           'FIPS_CTY': 'uint8', # FIPS County code
           'CPOPCRIM': 'uint32', # County population - agencies report crime
           'AG_OFF' : 'uint32', # No. Agencies in county report crime
           'COVIND': 'float64', # Coverage Indicator: proportion of the county's population for which full-year data is available.
           'VIOL': 'uint64', # Total violent crimes
           'PROPERTY': 'uint64'} # Total property crimes


In [50]:
ucrp2016 = pd.read_csv('UCRP_2016_CrimesReported.tsv', usecols=lambda x: x.upper() in columns.keys(), dtype=columns, sep='\t')
ucrp2016

,FIPS_ST,FIPS_CTY,CPOPCRIM,AG_OFF,COVIND,VIOL,PROPERTY
0,1,1,54499,3,100.0000,153,1722
1,1,3,207584,14,99.5843,443,4057
2,1,5,25778,4,93.9483,114,664
3,1,7,22474,5,96.0087,16,206
4,1,9,57565,6,98.1104,433,1228
...,...,...,...,...,...,...,...
3174,56,39,10713,1,100.0000,30,195
3175,56,41,20587,2,100.0000,19,407
3176,56,43,8213,2,100.0000,6,76
3177,56,45,6070,2,100.0000,8,57


Original dataset has 3179 rows

In [51]:
ucrp2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3179 entries, 0 to 3178
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   FIPS_ST   3179 non-null   uint8  
 1   FIPS_CTY  3179 non-null   uint8  
 2   CPOPCRIM  3179 non-null   uint32 
 3   AG_OFF    3179 non-null   uint32 
 4   COVIND    3179 non-null   float64
 5   VIOL      3179 non-null   uint64 
 6   PROPERTY  3179 non-null   uint64 
dtypes: float64(1), uint32(2), uint64(2), uint8(2)
memory usage: 105.7 KB


In [52]:
ucrp2016.describe()

,FIPS_ST,FIPS_CTY,CPOPCRIM,AG_OFF,COVIND,VIOL,PROPERTY
count,3179.000000,3179.000000,3.179000e+03,3179.000000,3179.000000,3179.000000,3179.000000
mean,30.281535,82.727273,9.961337e+04,5.447625,95.229193,397.755898,2445.499528
std,15.156837,60.381524,3.285690e+05,7.587667,18.482526,1832.299283,9072.729040
min,1.000000,1.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,18.000000,31.000000,9.080500e+03,2.000000,100.000000,13.000000,101.000000
50%,29.000000,73.000000,2.348000e+04,3.000000,100.000000,51.000000,409.000000
75%,45.000000,123.000000,6.422600e+04,6.000000,100.000000,178.000000,1408.500000
max,56.000000,255.000000,1.017087e+07,136.000000,100.000000,56372.000000,252505.000000


### Handling Missing Values
Zeroes in violent and property crime counts can represent true zeroes or missing data. To distinguish between the two the codebook indicates that if the coverage indicator is greater than 0, then a zero observed in 'VIOL' or 'PROPERTY' are true zeroes.

In [53]:
# remove missing values or counties that have less than 3-months reporting
ucrp2016 = ucrp2016[ucrp2016['COVIND'] > 0].copy()
ucrp2016

,FIPS_ST,FIPS_CTY,CPOPCRIM,AG_OFF,COVIND,VIOL,PROPERTY
0,1,1,54499,3,100.0000,153,1722
1,1,3,207584,14,99.5843,443,4057
2,1,5,25778,4,93.9483,114,664
3,1,7,22474,5,96.0087,16,206
4,1,9,57565,6,98.1104,433,1228
...,...,...,...,...,...,...,...
3174,56,39,10713,1,100.0000,30,195
3175,56,41,20587,2,100.0000,19,407
3176,56,43,8213,2,100.0000,6,76
3177,56,45,6070,2,100.0000,8,57


In [54]:
ucrp2016.describe()

,FIPS_ST,FIPS_CTY,CPOPCRIM,AG_OFF,COVIND,VIOL,PROPERTY
count,3080.000000,3080.000000,3.080000e+03,3080.000000,3080.000000,3080.000000,3080.000000
mean,30.341558,82.859091,1.017419e+05,5.618831,98.290131,404.942532,2507.050649
std,15.233344,60.578361,3.308415e+05,7.646809,7.185383,1844.798767,9189.243322
min,1.000000,1.000000,0.000000e+00,1.000000,18.438900,0.000000,0.000000
25%,18.000000,31.000000,1.002950e+04,2.000000,100.000000,15.000000,114.000000
50%,29.500000,73.000000,2.468500e+04,3.000000,100.000000,55.000000,430.000000
75%,46.000000,123.000000,6.707050e+04,6.000000,100.000000,186.250000,1459.500000
max,56.000000,255.000000,1.017087e+07,136.000000,100.000000,56372.000000,252505.000000


In [58]:
# check for duplicate counties, dup has cpop 0 means special agency in the county
dup = ucrp2016[ucrp2016.duplicated(subset=['FIPS_ST','FIPS_CTY'],keep=False)]
ucrp2016 = ucrp2016.groupby(['FIPS_ST','FIPS_CTY']).agg({
    'AG_OFF': 'sum',
    'CPOPCRIM': 'sum',
    'VIOL': 'sum',
    'PROPERTY': 'sum',
    'COVIND': 'mean'
}).reset_index()


In [59]:
ucrp2016 #187

,FIPS_ST,FIPS_CTY,AG_OFF,CPOPCRIM,VIOL,PROPERTY,COVIND
0,1,1,3,54499,153,1722,100.0000
1,1,3,14,207584,443,4057,99.5843
2,1,5,4,25778,114,664,93.9483
3,1,7,5,22474,16,206,96.0087
4,1,9,7,57565,446,1593,99.0552
...,...,...,...,...,...,...,...
2887,56,37,3,44449,136,734,100.0000
2888,56,39,1,10713,30,195,100.0000
2889,56,41,2,20587,19,407,100.0000
2890,56,43,2,8213,6,76,100.0000


In [60]:
# Calculate total Type I Crime in reported in County
ucrp2016['T_CRIME'] = ucrp2016['VIOL'] + ucrp2016['PROPERTY']

In [61]:
# note: used ChatGPT to quickly get these
fips_to_abbr = {
    1: 'AL', 2: 'AK', 4: 'AZ', 5: 'AR', 6: 'CA',
    8: 'CO', 9: 'CT', 10: 'DE', 12: 'FL', 13: 'GA',
    15: 'HI', 16: 'ID', 17: 'IL', 18: 'IN', 19: 'IA',
    20: 'KS', 21: 'KY', 22: 'LA', 23: 'ME', 24: 'MD',
    25: 'MA', 26: 'MI', 27: 'MN', 28: 'MS', 29: 'MO',
    30: 'MT', 31: 'NE', 32: 'NV', 33: 'NH', 34: 'NJ',
    35: 'NM', 36: 'NY', 37: 'NC', 38: 'ND', 39: 'OH',
    40: 'OK', 41: 'OR', 42: 'PA', 44: 'RI', 45: 'SC',
    46: 'SD', 47: 'TN', 48: 'TX', 49: 'UT', 50: 'VT',
    51: 'VA', 53: 'WA', 54: 'WV', 55: 'WI', 56: 'WY'
}

In [62]:
ucrp2016['STATE'] = ucrp2016['FIPS_ST'].map(fips_to_abbr)
ucrp2016

,FIPS_ST,FIPS_CTY,AG_OFF,CPOPCRIM,VIOL,PROPERTY,COVIND,T_CRIME,STATE
0,1,1,3,54499,153,1722,100.0000,1875,AL
1,1,3,14,207584,443,4057,99.5843,4500,AL
2,1,5,4,25778,114,664,93.9483,778,AL
3,1,7,5,22474,16,206,96.0087,222,AL
4,1,9,7,57565,446,1593,99.0552,2039,AL
...,...,...,...,...,...,...,...,...,...
2887,56,37,3,44449,136,734,100.0000,870,WY
2888,56,39,1,10713,30,195,100.0000,225,WY
2889,56,41,2,20587,19,407,100.0000,426,WY
2890,56,43,2,8213,6,76,100.0000,82,WY


### **Allocating State-level Reported Offences**

In [72]:
s_columns = {'FIPS_ST': 'uint8', # FIPS State code
           'FIPS_CTY': 'uint8', # FIPS County code
           'COVIND': 'float64', # Coverage Indicator: proportion of the county's population for which full-year data is available.
           'SVIOL': 'uint64', # Total violent crimes
           'SPROPERTY': 'uint64'} # Total property crimes

In [73]:
ucrp2016_s = pd.read_csv('UCRP_2016_StateAllocated.tsv', usecols=lambda x: x.upper() in s_columns.keys(), dtype=columns, sep='\t')

In [74]:
ucrp2016_s

,FIPS_ST,FIPS_CTY,COVIND,SVIOL,SPROPERTY
0,1,1,100.0000,0,0
1,1,3,99.5843,0,0
2,1,5,93.9483,0,0
3,1,7,96.0087,0,0
4,1,9,98.1104,0,0
...,...,...,...,...,...
3174,56,39,100.0000,0,0
3175,56,41,100.0000,0,0
3176,56,43,100.0000,0,0
3177,56,45,100.0000,0,0


In [75]:
ucrp2016_s = ucrp2016_s[ucrp2016_s['COVIND'] > 0].copy()

In [84]:
ucrp2016_s['T_CRIME'] = ucrp2016_s['SVIOL'] + ucrp2016_s['SPROPERTY']
ucrp2016_s.drop(columns = ['COVIND', 'SVIOL','SPROPERTY'], inplace=True)

In [86]:
dup = ucrp2016_s[ucrp2016_s.duplicated(subset=['FIPS_ST','FIPS_CTY'],keep=False)]
dup[dup['T_CRIME'] > 0]

,FIPS_ST,FIPS_CTY,T_CRIME
901,20,9,1
1821,35,9,23
1854,36,9,2
2342,44,9,18
2892,51,83,1
2933,51,171,1
2934,51,173,1
2967,51,171,1
2979,51,7,1


In [88]:
ucrp2016_s.groupby(['FIPS_ST','FIPS_CTY']).sum().reset_index()

,FIPS_ST,FIPS_CTY,T_CRIME
0,1,1,0
1,1,3,0
2,1,5,0
3,1,7,0
4,1,9,0
...,...,...,...
2887,56,37,0
2888,56,39,0
2889,56,41,0
2890,56,43,0


In [90]:
ucrp2016_merge = pd.merge(ucrp2016,ucrp2016_s, on=['FIPS_ST','FIPS_CTY'], how='left', suffixes=(None,'_S'))
ucrp2016_merge['T_CRIME'] = ucrp2016_merge['T_CRIME'] + ucrp2016_merge['T_CRIME_S']
ucrp2016_merge

,FIPS_ST,FIPS_CTY,AG_OFF,CPOPCRIM,VIOL,PROPERTY,COVIND,T_CRIME,STATE,T_CRIME_S
0,1,1,3,54499,153,1722,100.0000,1875.0,AL,0
1,1,3,14,207584,443,4057,99.5843,4500.0,AL,0
2,1,5,4,25778,114,664,93.9483,778.0,AL,0
3,1,7,5,22474,16,206,96.0087,222.0,AL,0
4,1,9,7,57565,446,1593,99.0552,2039.0,AL,0
...,...,...,...,...,...,...,...,...,...,...
3075,56,37,3,44449,136,734,100.0000,870.0,WY,0
3076,56,39,1,10713,30,195,100.0000,225.0,WY,0
3077,56,41,2,20587,19,407,100.0000,426.0,WY,0
3078,56,43,2,8213,6,76,100.0000,82.0,WY,0


In [41]:
ucrp2016[(ucrp2016['FIPS_ST'] == 1) & (ucrp2016['FIPS_CTY'] == 1)]

,FIPS_ST,FIPS_CTY,CPOPCRIM,AG_OFF,COVIND,VIOL,PROPERTY,STATE
0,1,1,54499,3,100.0,153,1722,AL


### Calculating Total Crime by State * will validate later no longer using coverage

In [152]:
ucrp2016.loc[:, 'TOTAL_CRIME'] = ucrp2016['VIOL'] + ucrp2016['PROPERTY']
#ucrp2016.loc[:, 'ADJ_TOTAL_CRIME'] = ucrp2016['TOTAL_CRIME'] / (ucrp2016['COVIND'] / 100)

crimes_by_state_2016 = ucrp2016.groupby('STATE')[['TOTAL_CRIME','AG_OFF','CPOPCRIM']].sum().reset_index()
crimes_by_state_2016['YEAR'] = 2016
crimes_by_state_2016 = crimes_by_state_2016.sort_values('STATE').reset_index(drop=True)


In [153]:
crimes_by_state_2016[crimes_by_state_2016['STATE'] == 'PA']

,STATE,TOTAL_CRIME,AG_OFF,CPOPCRIM,YEAR
37,PA,261931,1508,12732372,2016


## **Uniform Crime Reporting Program Data: Agency-Level Detailed Offense Data 2020**

In [134]:
rename_cols = {
    'V2': 'STATE_CODE', # numeric state code that is not FIP
    'V3': 'ORI',
    'V4': 'GROUP_NUM',
    'V12': 'NUM_MONTH', # number of months reported (last month reported)
    'V14': 'POP1',
    'V15': 'COUNTY1',
    'V17': 'POP2',
    'V18': 'COUNTY2',
    'V20': 'POP3',
    'V21': 'COUNTY3',

    'V36': 'JAN_TYPE',
    'V85': 'JAN_SIMPLE_ASSLT',
    'V95': 'JAN_ALL_FIELDS',

    'V154': 'FEB_TYPE',
    'V203': 'FEB_SIMPLE_ASSLT',
    'V213': 'FEB_ALL_FIELDS',

    'V272': 'MAR_TYPE',
    'V321': 'MAR_SIMPLE_ASSLT',
    'V331': 'MAR_ALL_FIELDS',

    'V390': 'APR_TYPE',
    'V439': 'APR_SIMPLE_ASSLT',
    'V449': 'APR_ALL_FIELDS',

    'V508': 'MAY_TYPE',
    'V557': 'MAY_SIMPLE_ASSLT',
    'V567': 'MAY_ALL_FIELDS',

    'V626': 'JUN_TYPE',
    'V675': 'JUN_SIMPLE_ASSLT',
    'V685': 'JUN_ALL_FIELDS',

    'V744': 'JUL_TYPE',
    'V793': 'JUL_SIMPLE_ASSLT',
    'V803': 'JUL_ALL_FIELDS',

    'V862': 'AUG_TYPE',
    'V911': 'AUG_SIMPLE_ASSLT',
    'V921': 'AUG_ALL_FIELDS',

    'V980': 'SEP_TYPE',
    'V1029': 'SEP_SIMPLE_ASSLT',
    'V1039': 'SEP_ALL_FIELDS',

    'V1098': 'OCT_TYPE',
    'V1147': 'OCT_SIMPLE_ASSLT',
    'V1157': 'OCT_ALL_FIELDS',

    'V1216': 'NOV_TYPE',
    'V1265': 'NOV_SIMPLE_ASSLT',
    'V1275': 'NOV_ALL_FIELDS',

    'V1334': 'DEC_TYPE',
    'V1383': 'DEC_SIMPLE_ASSLT',
    'V1393': 'DEC_ALL_FIELDS'
}

dtype = {
    col.upper(): ('string' if col.upper() in ['V3','V4'] else 'Int64')
    for col in rename_cols.keys()
}

In [135]:
ucrp2020 = pd.read_csv('UCRP_2020.tsv', usecols=lambda x: x.upper() in rename_cols.keys(), dtype = dtype, sep='\t', na_values=[" "])
ucrp2020 = ucrp2020.rename(columns=rename_cols)
ucrp2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24620 entries, 0 to 24619
Data columns (total 46 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   STATE_CODE        24620 non-null  Int64 
 1   ORI               24620 non-null  string
 2   GROUP_NUM         24620 non-null  string
 3   NUM_MONTH         24620 non-null  Int64 
 4   POP1              24620 non-null  Int64 
 5   COUNTY1           24620 non-null  Int64 
 6   POP2              24620 non-null  Int64 
 7   COUNTY2           680 non-null    Int64 
 8   POP3              24620 non-null  Int64 
 9   COUNTY3           60 non-null     Int64 
 10  JAN_TYPE          24620 non-null  Int64 
 11  JAN_SIMPLE_ASSLT  24620 non-null  Int64 
 12  JAN_ALL_FIELDS    24620 non-null  Int64 
 13  FEB_TYPE          24620 non-null  Int64 
 14  FEB_SIMPLE_ASSLT  24620 non-null  Int64 
 15  FEB_ALL_FIELDS    24620 non-null  Int64 
 16  MAR_TYPE          24620 non-null  Int64 
 17  MAR_SIMPLE_A

In [136]:
ucrp2020.describe()

,STATE_CODE,NUM_MONTH,POP1,COUNTY1,POP2,COUNTY2,POP3,COUNTY3,JAN_TYPE,JAN_SIMPLE_ASSLT,...,SEP_ALL_FIELDS,OCT_TYPE,OCT_SIMPLE_ASSLT,OCT_ALL_FIELDS,NOV_TYPE,NOV_SIMPLE_ASSLT,NOV_ALL_FIELDS,DEC_TYPE,DEC_SIMPLE_ASSLT,DEC_ALL_FIELDS
count,24620.0,24620.0,24620.0,24620.0,24620.0,680.0,24620.0,60.0,24620.0,24620.0,...,24620.0,24620.0,24620.0,24620.0,24620.0,24620.0,24620.0,24620.0,24620.0,24620.0
mean,26.519334,6.977051,13667.412632,37.839155,80.651868,53.714706,8.122908,61.866667,4.568684,7.88489,...,30.737693,4.521893,7.580301,31.324127,4.505768,6.9171,29.180707,4.497035,13.51974,55.369415
std,15.77861,5.594086,84522.90173,39.174689,1755.48241,45.642987,552.86034,46.087782,0.49527,57.86316,...,253.741204,0.499531,61.564495,263.297538,0.499977,56.655599,246.980581,0.500001,152.18307,613.002625
min,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,4.0,0.0,...,-4.0,4.0,0.0,0.0,4.0,-7.0,-7.0,4.0,-1.0,-6.0
25%,13.0,0.0,0.0,9.0,0.0,22.0,0.0,25.25,4.0,0.0,...,0.0,4.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0
50%,28.0,11.0,1710.0,29.0,0.0,44.5,0.0,60.0,5.0,0.0,...,0.0,5.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0
75%,37.0,12.0,8643.25,56.0,0.0,71.0,0.0,82.25,5.0,3.0,...,10.0,5.0,2.0,10.0,5.0,2.0,8.0,5.0,2.0,9.0
max,99.0,12.0,8300377.0,999.0,129948.0,247.0,60219.0,237.0,5.0,3567.0,...,16300.0,5.0,4201.0,16553.0,5.0,3874.0,15269.0,5.0,11459.0,44320.0


* High null values in POP2, COUNTY2, POP3, COUNTY3 are as expected because some city agencies are located
in as many as three different counties
* All return_types for each month are either 4 (not reported) / 5 (normal return)
* For some months, there are negative crimes reported by an ORI, due to updates offense being cleared

In [137]:
# keep only 50 states + District of Columbia
ucrp2020 = ucrp2020[ucrp2020['STATE_CODE'] <= 51].copy()
ucrp2020

,STATE_CODE,ORI,GROUP_NUM,NUM_MONTH,POP1,COUNTY1,POP2,COUNTY2,POP3,COUNTY3,...,SEP_ALL_FIELDS,OCT_TYPE,OCT_SIMPLE_ASSLT,OCT_ALL_FIELDS,NOV_TYPE,NOV_SIMPLE_ASSLT,NOV_ALL_FIELDS,DEC_TYPE,DEC_SIMPLE_ASSLT,DEC_ALL_FIELDS
0,50,AK00101,1C,12,286388,0,0,<NA>,0,<NA>,...,1479,5,252,1521,5,262,1254,5,287,1296
1,50,AK00102,4,12,30832,0,0,<NA>,0,<NA>,...,183,5,38,178,5,29,162,5,40,157
2,50,AK00103,4,12,31925,0,0,<NA>,0,<NA>,...,149,5,56,221,5,36,116,5,42,125
3,50,AK00104,6,12,8307,0,0,<NA>,0,<NA>,...,43,5,5,17,5,11,30,5,3,28
4,50,AK00105,6,12,5778,0,0,<NA>,0,<NA>,...,16,5,11,22,5,24,34,5,6,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24615,49,WYDI005,7,0,0,0,0,<NA>,0,<NA>,...,0,4,0,0,4,0,0,4,0,0
24616,49,WYDI006,7,0,0,0,0,<NA>,0,<NA>,...,0,4,0,0,4,0,0,4,0,0
24617,49,WYDI050,7,12,0,0,0,<NA>,0,<NA>,...,34,5,12,38,5,13,33,5,15,32
24618,49,WYWHP00,8D,0,0,0,0,<NA>,0,<NA>,...,0,4,0,0,4,0,0,4,0,0


In [138]:
# note: used ChatGPT to quickly get this dictionary from the documentation's V2: NUMERIC STATE CODE description.
state_code_to_abbrev = {
    1: 'AL',  # Alabama
    2: 'AZ',  # Arizona
    3: 'AR',  # Arkansas
    4: 'CA',  # California
    5: 'CO',  # Colorado
    6: 'CT',  # Connecticut
    7: 'DE',  # Delaware
    8: 'DC',  # District of Columbia
    9: 'FL',  # Florida
    10: 'GA', # Georgia
    11: 'ID', # Idaho
    12: 'IL', # Illinois
    13: 'IN', # Indiana
    14: 'IA', # Iowa
    15: 'KS', # Kansas
    16: 'KY', # Kentucky
    17: 'LA', # Louisiana
    18: 'ME', # Maine
    19: 'MD', # Maryland
    20: 'MA', # Massachusetts
    21: 'MI', # Michigan
    22: 'MN', # Minnesota
    23: 'MS', # Mississippi
    24: 'MO', # Missouri
    25: 'MT', # Montana
    26: 'NE', # Nebraska
    27: 'NV', # Nevada
    28: 'NH', # New Hampshire
    29: 'NJ', # New Jersey
    30: 'NM', # New Mexico
    31: 'NY', # New York
    32: 'NC', # North Carolina
    33: 'ND', # North Dakota
    34: 'OH', # Ohio
    35: 'OK', # Oklahoma
    36: 'OR', # Oregon
    37: 'PA', # Pennsylvania
    38: 'RI', # Rhode Island
    39: 'SC', # South Carolina
    40: 'SD', # South Dakota
    41: 'TN', # Tennessee
    42: 'TX', # Texas
    43: 'UT', # Utah
    44: 'VT', # Vermont
    45: 'VA', # Virginia
    46: 'WA', # Washington
    47: 'WV', # West Virginia
    48: 'WI', # Wisconsin
    49: 'WY', # Wyoming
    50: 'AK', # Alaska
    51: 'HI', # Hawaii
}

In [139]:
ucrp2020['STATE'] = ucrp2020['STATE_CODE'].map(state_code_to_abbrev)
ucrp2020.drop('STATE_CODE', axis=1, inplace=True)
ucrp2020 = ucrp2020[['STATE'] + ucrp2020.columns.drop('STATE').tolist()].copy()

In [140]:
# clean group num
ucrp2020['GROUP_NUM'] = ucrp2020['GROUP_NUM'].str.strip().str.extract(r'^(\d+)').astype('Int8')
ucrp2020['GROUP_NUM'].unique()

<IntegerArray>
[1, 4, 6, 7, 5, 8, 9, 2, 3]
Length: 9, dtype: Int8

### Calculating Total Crimes Reported by Month
The Uniform Crime Reporting Program Data: County-Level Detailed Arrest and Offense Data, United States, 2016 reports counts of offenses for UCR Part I crimes: murder, rape, robbery, aggravated assault, burglary, larceny, auto theft, and arson.

In the 2020 data, these same crimes are reported, but the total offense count (*_ALL_FIELDS*) also includes simple assault. To maintain consistency with the 2016 methodology, simple assault counts (*_SIMPLE_ASSLT*) are subtracted from the total to isolate only UCR Part I offenses. The result is stored as a new {month}_TCRIME column for each month then summed to get the total number of crimes reported by that agency in 2020.

In [141]:
months = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN',
          'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']

for month in months:
    ucrp2020.loc[:, f'{month}_TCRIME'] = ucrp2020[f'{month}_ALL_FIELDS'] - ucrp2020[f'{month}_SIMPLE_ASSLT']


In [142]:
tcrime_cols = ucrp2020.filter(regex='_TCRIME$').columns
ucrp2020['TCRIME_ADJ'] = ucrp2020[tcrime_cols].sum(axis=1)
ucrp2020.drop(columns= tcrime_cols, inplace = True)

In [143]:
ucrp2020

,STATE,ORI,GROUP_NUM,NUM_MONTH,POP1,COUNTY1,POP2,COUNTY2,POP3,COUNTY3,...,OCT_TYPE,OCT_SIMPLE_ASSLT,OCT_ALL_FIELDS,NOV_TYPE,NOV_SIMPLE_ASSLT,NOV_ALL_FIELDS,DEC_TYPE,DEC_SIMPLE_ASSLT,DEC_ALL_FIELDS,TCRIME_ADJ
0,AK,AK00101,1,12,286388,0,0,<NA>,0,<NA>,...,5,252,1521,5,262,1254,5,287,1296,13344
1,AK,AK00102,4,12,30832,0,0,<NA>,0,<NA>,...,5,38,178,5,29,162,5,40,157,1536
2,AK,AK00103,4,12,31925,0,0,<NA>,0,<NA>,...,5,56,221,5,36,116,5,42,125,1242
3,AK,AK00104,6,12,8307,0,0,<NA>,0,<NA>,...,5,5,17,5,11,30,5,3,28,287
4,AK,AK00105,6,12,5778,0,0,<NA>,0,<NA>,...,5,11,22,5,24,34,5,6,19,124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24615,WY,WYDI005,7,0,0,0,0,<NA>,0,<NA>,...,4,0,0,4,0,0,4,0,0,0
24616,WY,WYDI006,7,0,0,0,0,<NA>,0,<NA>,...,4,0,0,4,0,0,4,0,0,0
24617,WY,WYDI050,7,12,0,0,0,<NA>,0,<NA>,...,5,12,38,5,13,33,5,15,32,268
24618,WY,WYWHP00,8,0,0,0,0,<NA>,0,<NA>,...,4,0,0,4,0,0,4,0,0,0


In [144]:
tcrime_cols = ucrp2020.filter(regex='_FIELDS$').columns
ucrp2020['TCRIME'] = ucrp2020[tcrime_cols].sum(axis=1)
ucrp2020.drop(columns= tcrime_cols, inplace = True)

In [145]:
ucrp2020

,STATE,ORI,GROUP_NUM,NUM_MONTH,POP1,COUNTY1,POP2,COUNTY2,POP3,COUNTY3,...,SEP_TYPE,SEP_SIMPLE_ASSLT,OCT_TYPE,OCT_SIMPLE_ASSLT,NOV_TYPE,NOV_SIMPLE_ASSLT,DEC_TYPE,DEC_SIMPLE_ASSLT,TCRIME_ADJ,TCRIME
0,AK,AK00101,1,12,286388,0,0,<NA>,0,<NA>,...,5,267,5,252,5,262,5,287,13344,16783
1,AK,AK00102,4,12,30832,0,0,<NA>,0,<NA>,...,5,32,5,38,5,29,5,40,1536,1994
2,AK,AK00103,4,12,31925,0,0,<NA>,0,<NA>,...,5,56,5,56,5,36,5,42,1242,1899
3,AK,AK00104,6,12,8307,0,0,<NA>,0,<NA>,...,5,10,5,5,5,11,5,3,287,373
4,AK,AK00105,6,12,5778,0,0,<NA>,0,<NA>,...,5,5,5,11,5,24,5,6,124,277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24615,WY,WYDI005,7,0,0,0,0,<NA>,0,<NA>,...,4,0,4,0,4,0,4,0,0,0
24616,WY,WYDI006,7,0,0,0,0,<NA>,0,<NA>,...,4,0,4,0,4,0,4,0,0,0
24617,WY,WYDI050,7,12,0,0,0,<NA>,0,<NA>,...,5,18,5,12,5,13,5,15,268,472
24618,WY,WYWHP00,8,0,0,0,0,<NA>,0,<NA>,...,4,0,4,0,4,0,4,0,0,0


In [146]:
ucrp2020.describe()

,GROUP_NUM,NUM_MONTH,POP1,COUNTY1,POP2,COUNTY2,POP3,COUNTY3,JAN_TYPE,JAN_SIMPLE_ASSLT,...,SEP_TYPE,SEP_SIMPLE_ASSLT,OCT_TYPE,OCT_SIMPLE_ASSLT,NOV_TYPE,NOV_SIMPLE_ASSLT,DEC_TYPE,DEC_SIMPLE_ASSLT,TCRIME_ADJ,TCRIME
count,24276.0,24276.0,24276.0,24276.0,24276.0,680.0,24276.0,60.0,24276.0,24276.0,...,24276.0,24276.0,24276.0,24276.0,24276.0,24276.0,24276.0,24276.0,24276.0,24276.0
mean,6.802768,7.063973,13730.94233,38.327731,81.794736,53.714706,8.238013,61.866667,4.576413,7.995716,...,4.535467,7.718405,4.528959,7.687346,4.512729,7.014994,4.503666,13.711031,293.384783,393.454605
std,1.481776,5.574917,82674.567849,38.700333,1767.850647,45.642987,556.762988,46.087782,0.494137,58.2641,...,0.498751,61.928394,0.499171,61.992548,0.499848,57.049606,0.499997,153.249019,2389.86031,2996.723942
min,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,4.0,0.0,...,4.0,-9.0,4.0,0.0,4.0,-7.0,4.0,-1.0,0.0,0.0
25%,6.0,0.0,0.0,10.0,0.0,22.0,0.0,25.25,4.0,0.0,...,4.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0
50%,7.0,11.0,1795.0,30.0,0.0,44.5,0.0,60.0,5.0,0.0,...,5.0,0.0,5.0,0.0,5.0,0.0,5.0,0.0,9.0,13.0
75%,8.0,12.0,8839.75,56.0,0.0,71.0,0.0,82.25,5.0,3.0,...,5.0,2.0,5.0,2.0,5.0,2.0,5.0,2.0,103.0,144.0
max,9.0,12.0,8300377.0,314.0,129948.0,247.0,60219.0,237.0,5.0,3567.0,...,5.0,4032.0,5.0,4201.0,5.0,3874.0,5.0,11459.0,177321.0,177321.0


### Replicating 2016 Imputation Procedure
* Agencies that reported 0 months are dropped.
* Agencies reporting between 3 and 11 months have their crimes multiplied by 12/number of months reported. Such that an agency that reports only 6 months out of the year and reported 10 crimes in total would be estimated to have had 20 Part I crimes in that year (10 crimes reported * 12/6 months reported = 10 * 2 = 20).
* Agencies reporting 1-2 months would simply have the average (mean) number of crimes for agencies in that state and population group (e.g. cities population 250,000+, cities population 10,000-24,999). This average is generated only by agencies that reported all 12 months of the year. Such that if an agency reported 15 crimes and only reported 2 months of the year, that agency would get the average number of murders for similar sized agencies (same population group) in that state during that year.


To better understand the extent of monthly reporting, we create a new column `MONTH_REP` that counts how many months (out of 12) have a `{month}_TYPE` value of 5 ('Normal Return'). This helps quantify the completeness of monthly data submission by each agency.


In [147]:
# remove columns that did not report offences to UCR for any month
type_cols = ucrp2020.filter(regex='_TYPE$').columns
# ucrp2020 = ucrp2020[~(ucrp2020[type_cols] == 4).all(axis=1)]
ucrp2020.loc[:, 'MONTH_REP'] = (ucrp2020[type_cols] == 5).sum(axis=1)
ucrp2020.drop(columns = type_cols, inplace = True)
ucrp2020

,STATE,ORI,GROUP_NUM,NUM_MONTH,POP1,COUNTY1,POP2,COUNTY2,POP3,COUNTY3,...,JUN_SIMPLE_ASSLT,JUL_SIMPLE_ASSLT,AUG_SIMPLE_ASSLT,SEP_SIMPLE_ASSLT,OCT_SIMPLE_ASSLT,NOV_SIMPLE_ASSLT,DEC_SIMPLE_ASSLT,TCRIME_ADJ,TCRIME,MONTH_REP
0,AK,AK00101,1,12,286388,0,0,<NA>,0,<NA>,...,329,304,311,267,252,262,287,13344,16783,12
1,AK,AK00102,4,12,30832,0,0,<NA>,0,<NA>,...,66,56,36,32,38,29,40,1536,1994,12
2,AK,AK00103,4,12,31925,0,0,<NA>,0,<NA>,...,39,42,71,56,56,36,42,1242,1899,12
3,AK,AK00104,6,12,8307,0,0,<NA>,0,<NA>,...,10,3,8,10,5,11,3,287,373,12
4,AK,AK00105,6,12,5778,0,0,<NA>,0,<NA>,...,11,13,20,5,11,24,6,124,277,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24615,WY,WYDI005,7,0,0,0,0,<NA>,0,<NA>,...,0,0,0,0,0,0,0,0,0,0
24616,WY,WYDI006,7,0,0,0,0,<NA>,0,<NA>,...,0,0,0,0,0,0,0,0,0,0
24617,WY,WYDI050,7,12,0,0,0,<NA>,0,<NA>,...,41,25,24,18,12,13,15,268,472,12
24618,WY,WYWHP00,8,0,0,0,0,<NA>,0,<NA>,...,0,0,0,0,0,0,0,0,0,0


In [240]:
# checking if there are crimes reported when agency has not reported crimes any of the months just in case (since calculated based of flag not nan values in ALL_FIELDS columns)
ucrp2020[(ucrp2020['MONTH_REP']==0) & (ucrp2020['TCRIME'] > 0)]

,STATE,ORI,GROUP_NUM,POP1,COUNTY1,POP2,COUNTY2,POP3,COUNTY3,TCRIME,MONTH_REP


In [149]:
#
ucrp2020[ucrp2020['MONTH_REP'] == ucrp2020['NUM_MONTH']]

,STATE,ORI,GROUP_NUM,NUM_MONTH,POP1,COUNTY1,POP2,COUNTY2,POP3,COUNTY3,...,JUN_SIMPLE_ASSLT,JUL_SIMPLE_ASSLT,AUG_SIMPLE_ASSLT,SEP_SIMPLE_ASSLT,OCT_SIMPLE_ASSLT,NOV_SIMPLE_ASSLT,DEC_SIMPLE_ASSLT,TCRIME_ADJ,TCRIME,MONTH_REP
0,AK,AK00101,1,12,286388,0,0,<NA>,0,<NA>,...,329,304,311,267,252,262,287,13344,16783,12
1,AK,AK00102,4,12,30832,0,0,<NA>,0,<NA>,...,66,56,36,32,38,29,40,1536,1994,12
2,AK,AK00103,4,12,31925,0,0,<NA>,0,<NA>,...,39,42,71,56,56,36,42,1242,1899,12
3,AK,AK00104,6,12,8307,0,0,<NA>,0,<NA>,...,10,3,8,10,5,11,3,287,373,12
4,AK,AK00105,6,12,5778,0,0,<NA>,0,<NA>,...,11,13,20,5,11,24,6,124,277,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24615,WY,WYDI005,7,0,0,0,0,<NA>,0,<NA>,...,0,0,0,0,0,0,0,0,0,0
24616,WY,WYDI006,7,0,0,0,0,<NA>,0,<NA>,...,0,0,0,0,0,0,0,0,0,0
24617,WY,WYDI050,7,12,0,0,0,<NA>,0,<NA>,...,41,25,24,18,12,13,15,268,472,12
24618,WY,WYWHP00,8,0,0,0,0,<NA>,0,<NA>,...,0,0,0,0,0,0,0,0,0,0


In [150]:
ucrp2020 = ucrp2020[ucrp2020['MONTH_REP'] > 0].copy()
len(ucrp2020)

15366

In [151]:
def impute_tcrime(row, crime_column: str, df: pd.DataFrame):
    if pd.isna(row['MONTH_REP']) or pd.isna(row[crime_column]):
        return pd.NA

    elif row['MONTH_REP'] == 12:
        return row[crime_column]

    elif 12 > row['MONTH_REP'] >= 3:
        return row[crime_column] * (12 / row['MONTH_REP'])

    elif row['MONTH_REP'] < 3:
        # Filter matching agencies (same state and population group) with full 12-month data
        match_agencies = (
            (df['MONTH_REP'] == 12) &
            (df['STATE'] == row['STATE']) &
            (df['GROUP_NUM'] == row['GROUP_NUM'])
        )
        matching = df.loc[match_agencies, crime_column]
        return matching.mean() if not matching.empty else pd.NA


In [153]:
ucrp2020['TCRIME_EST'] = ucrp2020.apply(lambda row: impute_tcrime(row,'TCRIME', ucrp2020), axis=1)
ucrp2020['TCRIME_ADJ_EST'] = ucrp2020.apply(lambda row: impute_tcrime(row,'TCRIME_ADJ', ucrp2020), axis=1)

In [154]:
ucrp2020[]

,STATE,ORI,GROUP_NUM,NUM_MONTH,POP1,COUNTY1,POP2,COUNTY2,POP3,COUNTY3,...,AUG_SIMPLE_ASSLT,SEP_SIMPLE_ASSLT,OCT_SIMPLE_ASSLT,NOV_SIMPLE_ASSLT,DEC_SIMPLE_ASSLT,TCRIME_ADJ,TCRIME,MONTH_REP,TCRIME_EST,TCRIME_ADJ_EST
0,AK,AK00101,1,12,286388,0,0,<NA>,0,<NA>,...,311,267,252,262,287,13344,16783,12,16783,13344
1,AK,AK00102,4,12,30832,0,0,<NA>,0,<NA>,...,36,32,38,29,40,1536,1994,12,1994,1536
2,AK,AK00103,4,12,31925,0,0,<NA>,0,<NA>,...,71,56,56,36,42,1242,1899,12,1899,1242
3,AK,AK00104,6,12,8307,0,0,<NA>,0,<NA>,...,8,10,5,11,3,287,373,12,373,287
4,AK,AK00105,6,12,5778,0,0,<NA>,0,<NA>,...,20,5,11,24,6,124,277,12,277,124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24605,WY,WY02200,8,12,2778,22,0,<NA>,0,<NA>,...,0,0,1,2,2,9,21,9,28.0,12.0
24606,WY,WY02201,6,12,4975,22,0,<NA>,0,<NA>,...,5,0,4,2,0,35,75,12,75,35
24607,WY,WY02300,8,12,2485,23,0,<NA>,0,<NA>,...,0,0,0,0,0,3,3,12,3,3
24608,WY,WY02301,6,12,3383,23,0,<NA>,0,<NA>,...,1,2,2,0,2,51,68,12,68,51


TO DO
- drop monthly assaults earlier
- keep the original group column so i can get 8E(Non-MSA State Police) and 9E(MSA State Police) for when i need to do state allocations
- double check i didnt drop cols were population is 0 across the board earlier
- run impute crime with the NUM_MONTH too since this is technically the one FBI uses
- allocations:
    - agencies present across multiple counties: assign TCRIME to counties based of their population

In [244]:
# Chcking if there are rows where POP1 is 0, but either POP2 or POP3 are not 0
violations = ucrp2020[(ucrp2020['POP1'] == 0) & ((ucrp2020['POP2'] != 0) | (ucrp2020['POP3'] != 0))]

violations.head()

,STATE,ORI,GROUP_NUM,POP1,COUNTY1,POP2,COUNTY2,POP3,COUNTY3,TCRIME,MONTH_REP,TCRIME_IMPUTED


In [109]:
ucrp2020 = ucrp2020[~((ucrp2020['POP1']==0) & (ucrp2020['MONTH_REP']< 3))].copy()
ucrp2020

,STATE,ORI,GROUP_NUM,POP1,COUNTY1,POP2,COUNTY2,POP3,COUNTY3,TCRIME,MONTH_REP,TCRIME_IMPUTED
0,AK,AK00101,1C,286388,0,0,<NA>,0,<NA>,13344,12,13344.0
1,AK,AK00102,4,30832,0,0,<NA>,0,<NA>,1536,12,1536.0
2,AK,AK00103,4,31925,0,0,<NA>,0,<NA>,1242,12,1242.0
3,AK,AK00104,6,8307,0,0,<NA>,0,<NA>,287,12,287.0
4,AK,AK00105,6,5778,0,0,<NA>,0,<NA>,124,12,124.0
...,...,...,...,...,...,...,...,...,...,...,...,...
24605,WY,WY02200,8D,2778,22,0,<NA>,0,<NA>,9,9,12.0
24606,WY,WY02201,6,4975,22,0,<NA>,0,<NA>,35,12,35.0
24607,WY,WY02300,8D,2485,23,0,<NA>,0,<NA>,3,12,3.0
24608,WY,WY02301,6,3383,23,0,<NA>,0,<NA>,51,12,51.0


In [245]:
ucrp2020[ucrp2020['TCRIME_IMPUTED'].isnull()] # kept for coverage indicator

,STATE,ORI,GROUP_NUM,POP1,COUNTY1,POP2,COUNTY2,POP3,COUNTY3,TCRIME,MONTH_REP,TCRIME_IMPUTED
8933,MA,MA304SP,8,0,4,0,<NA>,0,<NA>,1,2,NaN
8939,MA,MA310SP,8,0,10,0,<NA>,0,<NA>,0,1,NaN
18099,PA,PA01403,3,58471,14,0,<NA>,0,<NA>,50,1,NaN
18370,PA,PA02337,3,82947,23,0,<NA>,0,<NA>,190,2,NaN
18400,PA,PA02503,3,52329,25,0,<NA>,0,<NA>,93,2,NaN
18536,PA,PA03504,3,76719,35,0,<NA>,0,<NA>,404,2,NaN
18577,PA,PA03605,3,59264,36,0,<NA>,0,<NA>,141,1,NaN
18854,PA,PA04601,3,55310,46,0,<NA>,0,<NA>,136,2,NaN


In [246]:
# Total population of the jurisdiction
ucrp2020['TOTAL_POP'] = ucrp2020[['POP1', 'POP2', 'POP3']].sum(axis=1)

In [247]:
state_summary = ucrp2020.groupby('STATE').agg(
    ORI_COUNT=('ORI', 'count'),
    STATE_POP=('TOTAL_POP', 'sum'),
    AVG_MONTH_REP=('MONTH_REP', 'mean'),
    TOTAL_CRIME=('TCRIME_IMPUTED','sum')
).reset_index()

state_summary['YEAR'] = 2020
state_summary = state_summary[['STATE', 'YEAR', 'ORI_COUNT', 'STATE_POP', 'AVG_MONTH_REP', 'TOTAL_CRIME']]
state_summary = state_summary.sort_values('STATE').reset_index(drop=True)

state_summary.describe()

,YEAR,ORI_COUNT,STATE_POP,AVG_MONTH_REP,TOTAL_CRIME
count,51.0,51.000000,51.0,51.0,5.100000e+01
mean,2020.0,301.294118,6145890.196078,10.449333,1.456308e+05
std,0.0,245.725480,7275927.392706,1.170454,1.817243e+05
min,2020.0,3.000000,573644.0,4.856707,9.671989e+03
25%,2020.0,110.500000,1681017.0,9.798567,3.488650e+04
50%,2020.0,234.000000,4339461.0,10.544484,9.575294e+04
75%,2020.0,408.000000,7285664.5,11.289,1.857683e+05
max,2020.0,1065.000000,39369978.0,12.0,1.015339e+06


In [253]:
state_summary[state_summary['AVG_MONTH_REP'] < 7]

,STATE,YEAR,ORI_COUNT,STATE_POP,AVG_MONTH_REP,TOTAL_CRIME
38,PA,2020,984,8184706,4.856707,146161.89027


In [62]:
import pandas as pd

In [73]:
raw_2016 = pd.read_excel('Data/Table_05_Crime_in_the_United_States_by_State_2020.xlsx', header= None)


In [74]:
import re

year = int(''.join(re.findall(r'\d+', str(raw_2016.iloc[2, 0]))))

In [76]:
# Step 1: Clean the header row (index 3)
cleaned_header = raw_2016.iloc[3].astype(str).str.replace('\n', ' ', regex=False).str.strip().str.replace(r'\d+', '', regex=True).str.upper().str.replace('  ', ' ')

# Step 2: Assign it as the header
raw_2016.columns = cleaned_header

# Step 3: Drop the first 4 rows (including the old header row)
raw_2016 = raw_2016.drop(index=range(4)).reset_index(drop=True)
raw_2016

3,STATE,AREA,NAN,POPULATION,VIOLENT CRIME,MURDER AND NONNEGLIGENT MANSLAUGHTER,RAPE (REVISED DEFINITION),ROBBERY,AGGRAVATED ASSAULT,PROPERTY CRIME,BURGLARY,LARCENY-THEFT,MOTOR VEHICLE THEFT
0,ALABAMA4,Metropolitan Statistical Area,NaN,3750044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Area actually reporting,0.866,16638,319,1147,2260,12912,75971,13921,53997,8053
2,NaN,NaN,Estimated total,1,17789,336,1243,2374,13836,84113,15452,59871,8790
3,NaN,Cities outside metropolitan areas,NaN,527001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,Area actually reporting,0.874,2834,73,188,204,2369,13407,2276,10061,1070
...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,5 Includes offenses reported by the Metro Tran...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
513,6 Because of changes in the state/local agency...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
514,7 The FBI determined that the state did not fo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
515,8 This state's agencies submitted rape data ac...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
raw_2016['STATE'].unique()

array(['ALABAMA4', nan, 'ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA',
       'COLORADO', 'CONNECTICUT', 'DELAWARE', 'DISTRICT OF COLUMBIA5',
       'FLORIDA', 'GEORGIA6', 'HAWAII', 'IDAHO', 'ILLINOIS7', 'INDIANA',
       'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND4',
       'MASSACHUSETTS', 'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI',
       'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEVADA', 'NEW HAMPSHIRE',
       'NEW JERSEY', 'NEW MEXICO', 'NEW YORK', 'NORTH CAROLINA', ' ',
       'NORTH DAKOTA', 'OHIO8', 'OKLAHOMA', 'OREGON8', 'PENNSYLVANIA4',
       'PUERTO RICO', 'RHODE ISLAND', 'SOUTH CAROLINA', 'SOUTH DAKOTA',
       'TENNESSEE', 'TEXAS', 'UTAH', 'VERMONT', 'VIRGINIA', 'WASHINGTON',
       'WEST VIRGINIA', 'WISCONSIN', 'WYOMING',
       '1  Population figures are U.S. Census Bureau provisional estimates as of July 1, 2020.',
       '2 The violent crime figures include the offenses of murder, rape (revised definition), robbery, and aggravated assault.',
       '3 The fig

In [79]:
# Clean the STATE column: replace all-whitespace entries with NaN
raw_2016['STATE'] = raw_2016['STATE'].replace(r'^\s*$', pd.NA, regex=True)
raw_2016['STATE']= raw_2016['STATE'].ffill()
raw_2016

3,STATE,AREA,NAN,POPULATION,VIOLENT CRIME,MURDER AND NONNEGLIGENT MANSLAUGHTER,RAPE (REVISED DEFINITION),ROBBERY,AGGRAVATED ASSAULT,PROPERTY CRIME,BURGLARY,LARCENY-THEFT,MOTOR VEHICLE THEFT
0,ALABAMA4,Metropolitan Statistical Area,NaN,3750044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALABAMA4,NaN,Area actually reporting,0.866,16638,319,1147,2260,12912,75971,13921,53997,8053
2,ALABAMA4,NaN,Estimated total,1,17789,336,1243,2374,13836,84113,15452,59871,8790
3,ALABAMA4,Cities outside metropolitan areas,NaN,527001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ALABAMA4,NaN,Area actually reporting,0.874,2834,73,188,204,2369,13407,2276,10061,1070
...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,5 Includes offenses reported by the Metro Tran...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
513,6 Because of changes in the state/local agency...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
514,7 The FBI determined that the state did not fo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
515,8 This state's agencies submitted rape data ac...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
raw_2016['STATE'].unique()

array(['ALABAMA4', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA',
       'COLORADO', 'CONNECTICUT', 'DELAWARE', 'DISTRICT OF COLUMBIA5',
       'FLORIDA', 'GEORGIA6', 'HAWAII', 'IDAHO', 'ILLINOIS7', 'INDIANA',
       'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND4',
       'MASSACHUSETTS', 'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI',
       'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEVADA', 'NEW HAMPSHIRE',
       'NEW JERSEY', 'NEW MEXICO', 'NEW YORK', 'NORTH CAROLINA', ' ',
       'NORTH DAKOTA', 'OHIO8', 'OKLAHOMA', 'OREGON8', 'PENNSYLVANIA4',
       'PUERTO RICO', 'RHODE ISLAND', 'SOUTH CAROLINA', 'SOUTH DAKOTA',
       'TENNESSEE', 'TEXAS', 'UTAH', 'VERMONT', 'VIRGINIA', 'WASHINGTON',
       'WEST VIRGINIA', 'WISCONSIN', 'WYOMING',
       '1  Population figures are U.S. Census Bureau provisional estimates as of July 1, 2020.',
       '2 The violent crime figures include the offenses of murder, rape (revised definition), robbery, and aggravated assault.',
       '3 The figures 

In [80]:
raw_2016 = raw_2016[raw_2016['AREA'].str.upper().str.endswith('TOTAL', na=False)].copy()
raw_2016['STATE'].unique()

array(['ALABAMA4', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA',
       'COLORADO', 'CONNECTICUT', 'DELAWARE', 'DISTRICT OF COLUMBIA5',
       'FLORIDA', 'GEORGIA6', 'HAWAII', 'IDAHO', 'ILLINOIS7', 'INDIANA',
       'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND4',
       'MASSACHUSETTS', 'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI',
       'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEVADA', 'NEW HAMPSHIRE',
       'NEW JERSEY', 'NEW MEXICO', 'NEW YORK', 'NORTH CAROLINA',
       'NORTH DAKOTA', 'OHIO8', 'OKLAHOMA', 'OREGON8', 'PENNSYLVANIA4',
       'PUERTO RICO', 'RHODE ISLAND', 'SOUTH CAROLINA', 'SOUTH DAKOTA',
       'TENNESSEE', 'TEXAS', 'UTAH', 'VERMONT', 'VIRGINIA', 'WASHINGTON',
       'WEST VIRGINIA', 'WISCONSIN', 'WYOMING'], dtype=object)

In [21]:
columns = {0:'STATE', 3:'POPULATION', 4:'V_CRIME', 10:'P_CRIME'}

crime_by_state = raw_2016[list(columns)].copy().rename(columns=columns)

crime_by_state['STATE'] =crime_by_state['STATE'].astype(str)
crime_by_state['STATE'] = crime_by_state['STATE'].str.strip().replace(r'\d+', '', regex=True)
crime_by_state = crime_by_state[~crime_by_state['STATE'].str.upper().eq('PUERTO RICO')]

state_to_abbrev = {
    'ALABAMA': 'AL', 'ALASKA': 'AK', 'ARIZONA': 'AZ', 'ARKANSAS': 'AR',
    'CALIFORNIA': 'CA', 'COLORADO': 'CO', 'CONNECTICUT': 'CT', 'DELAWARE': 'DE',
    'FLORIDA': 'FL', 'GEORGIA': 'GA', 'HAWAII': 'HI', 'IDAHO': 'ID',
    'ILLINOIS': 'IL', 'INDIANA': 'IN', 'IOWA': 'IA', 'KANSAS': 'KS',
    'KENTUCKY': 'KY', 'LOUISIANA': 'LA', 'MAINE': 'ME', 'MARYLAND': 'MD',
    'MASSACHUSETTS': 'MA', 'MICHIGAN': 'MI', 'MINNESOTA': 'MN', 'MISSISSIPPI': 'MS',
    'MISSOURI': 'MO', 'MONTANA': 'MT', 'NEBRASKA': 'NE', 'NEVADA': 'NV',
    'NEW HAMPSHIRE': 'NH', 'NEW JERSEY': 'NJ', 'NEW MEXICO': 'NM', 'NEW YORK': 'NY',
    'NORTH CAROLINA': 'NC', 'NORTH DAKOTA': 'ND', 'OHIO': 'OH', 'OKLAHOMA': 'OK',
    'OREGON': 'OR', 'PENNSYLVANIA': 'PA', 'RHODE ISLAND': 'RI', 'SOUTH CAROLINA': 'SC',
    'SOUTH DAKOTA': 'SD', 'TENNESSEE': 'TN', 'TEXAS': 'TX', 'UTAH': 'UT',
    'VERMONT': 'VT', 'VIRGINIA': 'VA', 'WASHINGTON': 'WA', 'WEST VIRGINIA': 'WV',
    'WISCONSIN': 'WI', 'WYOMING': 'WY', 'DISTRICT OF COLUMBIA': 'DC'
}

crime_by_state['STATE_CODE'] = crime_by_state['STATE'].str.upper().map(state_to_abbrev)

num_columns = list(columns.values())[1:]
crime_by_state[num_columns] = crime_by_state[num_columns].apply(pd.to_numeric, errors='coerce')

crime_by_state['TOTAL_CRIME'] = crime_by_state['V_CRIME'] + crime_by_state['P_CRIME']
crime_by_state.drop(num_columns[1:], axis=1, inplace=True)




In [91]:
crime_by_state

,STATE,POPULATION,TOTAL_CRIME
10,ALABAMA,4863300,169248
19,ALASKA,741894,30842
28,ARIZONA,6931071,239015
39,ARKANSAS,2988248,114134
48,CALIFORNIA,39250017,1176866
59,COLORADO,5540545,170833
67,CONNECTICUT,3576452,72787
74,DELAWARE,952065,31178
80,DISTRICT OF COLUMBIA,681170,40930
90,FLORIDA,20612439,642512


Using FBI data portal to verify certain ones; could not find releases of this aggregated data but useful for checking that aggregations arent crazy (esp after aggregation procedures, in 2020 I esp want to check alabama, pennsylvania for 2020)

“Area actually reporting” represent offense totals for agencies submitting 12 months of data and estimated totals for agencies submitting less than 12 but more than 2 months of data.

    “Estimated total” represent the totals under “Area actually reporting” plus estimated totals for agencies submitting 2 months or less of data.